# Group: Synergy

This code is used to create database on MongoDB

In [ ]:
from pymongo import MongoClient
import json
from bson.objectid import ObjectId
import numpy as np
import string
import pandas as pd

In [51]:
r1 = open('Output.json')
d1 = json.load(r1)

In [52]:
d2=[]
for k,v in d1.items():
    v["ISBN"]=k
    d2.append(v)

In [53]:
for i in range(len(d2)):
    d2[i]['author'] = d2[i]['author'].title()
    d2[i]['Name'] = string.capwords(d2[i]['Name'])
    d2[i]['Ratings']['average score'] = '%.1f' % (d2[i]['Ratings']['average score'])

In [55]:
client = MongoClient()

In [56]:
db = client.Project

In [57]:
books = db.books 

In [58]:
books.insert_many(d2)

In [59]:
rating=pd.read_csv("all_user_match.csv")

In [60]:
rating.columns=['User_ID','ISBN','Rating']

In [61]:
ratinglist=[]
for i in range(rating.shape[0]):
    ratinglist.append({"User_ID":str(rating.iloc[i,0]),"ISBN":rating.iloc[i,1],"Rating":str(rating.iloc[i,2])})

In [62]:
#db.getCollection("users").drop()

In [63]:
ratings = db.ratings 

In [64]:
ratings.insert_many(ratinglist)

In [65]:
user = pd.read_csv("users.csv", encoding="latin1")

In [66]:
del user['Unnamed: 0']
del user['Age']

In [67]:
user.columns = ['User_ID', 'Location']

In [68]:
result = pd.merge(user, rating, on='User_ID')

In [69]:
l = []
for i in range(result.shape[0]):
    if result.iloc[i, 1][-3:] != 'usa':
        l.append(i)
    else:
        result.iloc[i, 1] = result.iloc[i, 1].split(',')[1].strip()
result = result.drop(result.index[l])

In [70]:
result['region'] = ''

In [71]:
NE = ['maine', 'massachusetts', 'rhode island', 'connecticut', 'new hampshire',
      'vermont', 'new york', 'pennsylvania', 'new jersey', 'delaware', 'maryland']
SE = ['west virginia', 'virginia', 'kentucky', 'tennessee', 'north carolina',
      'south carolina', 'georgia', 'alabama', 'mississippi', 'arkansas', 'louisiana', 'florida']
MW = ['ohio', 'indiana', 'michigan', 'illinois', 'missouri', 'wisconsin',
      'minnesota', 'iowa', 'kansas', 'nebraska', 'south dakota', 'north dakota']
SW = ['texas', 'oklahoma', 'new mexico', 'arizona']
W = ['colorado', 'wyoming', 'montana', 'idaho', 'washington',
     'oregon', 'utah', 'nevada', 'california', 'alaska', 'hawaii']

In [72]:
for i in range(result.shape[0]):
    if result.iloc[i, 1] in NE:
        result.iloc[i, 4] = 'NouthEast'
    elif result.iloc[i, 1] in SE:
        result.iloc[i, 4] = 'SouthEast'
    elif result.iloc[i, 1] in MW:
        result.iloc[i, 4] = 'MidEast'
    elif result.iloc[i, 1] in SW:
        result.iloc[i, 4] = 'SouthWest'
    elif result.iloc[i, 1] in W:
        result.iloc[i, 4] = 'West'
    else:
        result.iloc[i, 4] = 'Other'

In [73]:
result = result.reset_index()

In [74]:
del result['index']

In [75]:
userlist = []
for i in range(result.shape[0]):
    userlist.append({"User_ID": str(result.iloc[i, 0]), "Location": result.iloc[
                    i, 1], "ISBN": result.iloc[i, 2], "Rating": str(result.iloc[i, 3]), "region": result.iloc[i, 4]})

In [76]:
users = db.users

In [77]:
users.insert_many(userlist)

In [81]:
def around(region, users, books):
    string = ""
    for i in users.find({'region': region}):
        isbn = i['ISBN'].zfill(10)
        tag = books.find_one({'ISBN': isbn})['tags']
        for j in tag:
            string += ' ' + j
    return string

# END